In [ ]:
import openai
import os
import json
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
import langchain
from langchain.embeddings import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
embedded_res = embeddings_model.embed_query("What was the name mentioned in the conversation?")
embedded_res[:5]

len(embedded_res)
# print (embeddings)

In [ ]:

import json
import pickle
import hashlib
hashlib.sha256(b"test text").hexdigest()
EMBEDDING_LEN = 1536 # OPENAI EMBEDDING LEN
kv_cache = pickle.load(open('kv_cache_openai_embedding.pickle', 'rb'))
def insert_embedding(data_file):
    # data_file = 'fp_dict_with_code_with_ir.json'
    data_file = "../smartagent-dataset/Scripts/gpt3.5_fp_with_code.json"
    fp_data = json.load(open(data_file))
    count = 0
    for key_,val_ in fp_data.items():
        for func_name, func_info in val_.items():
            print (func_name)
            if "code_embedding" in func_info:
                print ("found embedding ", len(func_info['code_embedding']))
                continue
            if 'code' in func_info:
                solidity_code = func_info['code'].get("raw")
                if (solidity_code is None) or (len(solidity_code) == 0):
                    continue
                # print (code)
                code_embedding = embeddings_model.embed_query(solidity_code)
                # print (hashlib.sha256(solidity_code.encode()).hexdigest())
                kv_cache[hashlib.sha256(solidity_code.encode()).hexdigest()] = code_embedding
                if code_embedding and len(code_embedding) != 0:
                    func_info['code_embedding'] = code_embedding
                    count += 1
        # if count > 5:
        #     break
        write_file = data_file.split('.')[0] + '_with_embedding_code.json'
        json.dump(fp_data, open(write_file, 'w'), indent=2)

insert_embedding('fp_dict_with_code_with_ir.json')

pickle.dump(kv_cache, open('kv_cache_openai_embedding.pickle', 'wb'))

# print (list(fp_data.values())[:10])



In [ ]:
# merge json files and remove ir
import json
file_1 = "gpt_fp_with_embedding_code.json"
file_2 = "fp_dict_with_code_ir_embedding_filtered.json"
final_file = "all_fp_data_with_embedding_merge.json"
with open(file_1) as f:
    data_1 = json.load(f)
with open(file_2) as f:
    data_2 = json.load(f)

all_fp_data = {}

for key_,val_ in file_1.items():
    new_val = {}
    for func_name, func_info in val_.items():
        if "code_embedding" not in func_info:
            print ("not found embedding ", func_name)
            continue
        else:
            new_val[func_name] = func_info
    if len(new_val) != 0:
        all_fp_data[key_] = new_val
for key_,val_ in file_2.items():
    if key_ in all_fp_data:
        new_val = all_fp_data[key_]
    else:
        new_val = {}
    for func_name, func_info in val_.items():
        if "code_embedding" not in func_info:
            print ("not found embedding ", func_name)
            continue
        else:
            new_val[func_name] = func_info
    if len(new_val) != 0:
        all_fp_data[key_] = new_val


json.dump(all_fp_data, open(final_file, 'w'), indent=2)


In [ ]:
# load test_data from pickle
import pickle
import json
# test_data = pickle.load(open('kv_cache_openai_embedding.pickle', 'rb'))
# print (list(test_data.values())[0])
# print (list(test_data.values())[1])
from sklearn.manifold import TSNE
from sklearn import preprocessing
from scipy import stats
from sklearn.metrics import silhouette_score
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from ast import literal_eval
tools = ['mythril', 'slither']
from sklearn.cluster import KMeans
# print (test_data)
from collections import Counter

def extract_tool_embedding(data_file, tool, embedding_type = 'code_embedding'):
    #data_file = 'fp_dict_with_code_with_ir_with_embedding_code_ir.json'
    fp_data = json.load(open(data_file))
    count = 0
    return_data = []
    for key_,val_ in fp_data.items():
        for func_name, func_info in val_.items():
            # print (func_name)
            if embedding_type in func_info and tool in func_info.get('tools', []):
                count = count + 1
                return_data.append(func_info.get(embedding_type))
    print ("found ", count, " functions with ", tool, embedding_type)
    return return_data
# res = extract_tool_embedding('fp_dict_with_code_with_ir_with_embedding_code_ir.json', 'slither', 'ir_embedding')

def plot_tool(tool_name, embedding_type, data_file ):
    data = extract_tool_embedding(data_file, tool_name, embedding_type)

    matrix = np.array(data)
    normalized_matrix =  preprocessing.normalize(matrix)

    if tool_name == 'mythril':
        n_clusters = 5
    elif tool_name == 'gpt_3.5':
        n_clusters = 5
    else:
        n_clusters = 5
    kmeans = KMeans(n_clusters=n_clusters, init="k-means++", random_state=42)
    kmeans.fit(normalized_matrix)
    labels = kmeans.labels_
    cluster_counts = Counter(labels)
    # print ("label data")
    for idx, item in enumerate(labels):
        # print (item)
        if item == 3:
            print (idx)
    # Print out the count of each cluster
    for cluster_id, count in cluster_counts.items():
        print(f"Cluster {cluster_id}: {count} items")

    # Step 1: Compute distances of each point to the centroids
    distances = kmeans.transform(normalized_matrix)

    # Step 2: Find the index of the closest point to each centroid
    closest_points_indices = np.argmin(distances, axis=0)

    # Extract the closest points
    # closest_points = normalized_matrix[closest_points_indices]
    print ("closest points ")
    print (closest_points_indices)
    print("data shape", matrix.shape)
    # print("data sample", matrix[:2][:5])
    # tsne = TSNE(n_components=2,metric="cosine", random_state=42, init="random")
    tsne = TSNE(n_components=2, perplexity=10,metric="cosine", random_state=42, init="random", learning_rate=200)
    vis_dims2 = tsne.fit_transform(matrix)
    x = [x for x, y in vis_dims2]
    y = [y for x, y in vis_dims2]
    print ("max min x idx ",np.argmax(x), np.argmin(x))
    print ("max min x val ",np.max(x), np.min(x))
    print ("max min y idx ",np.argmax(y), np.argmin(y))
    print ("max min y val ",np.max(y), np.min(y))
    plt.clf()
    plt.figure(figsize=(4*1.618,4))
    colormap = plt.cm.get_cmap('viridis', n_clusters)  # Replace 'viridis' with any other colormap if needed
    for category in range(n_clusters):
        color = colormap(category)
        # Correct the filtering here
        xs = np.array(x)[labels == category]
        ys = np.array(y)[labels == category]
        plt.scatter(xs, ys, color=color, alpha=0.3)

        if len(xs) > 0:
            avg_x = xs.mean()
            avg_y = ys.mean()
            plt.scatter(avg_x, avg_y, marker="x", color=color, s=100)

    # plt.title("Clusters visualization with t-SNE")
    plt.savefig(f"{tool_name}_{embedding_type}_cluster.pdf")
    return closest_points_indices

plot_tool('gpt_3.5', 'code_embedding', "merged_fp_all_tools.json")
plot_tool('slither', 'code_embedding', "merged_fp_all_tools.json")
plot_tool('mythril', 'code_embedding', "merged_fp_all_tools.json")



In [ ]:

# extract centroid representative code
def extract_centroid_code(data_file, tool, centroid_indices, embedding_type = 'code_embedding'):
    fp_data = json.load(open(data_file))
    count = 0
    return_data = []
    for key_,val_ in fp_data.items():
        for func_name, func_info in val_.items():
            # print (func_name)
            if embedding_type in func_info and tool in func_info.get('tools', []):
                if count in centroid_indices:
                    print ("found ", count, " functions with ", tool, embedding_type)
                    return_data.append({
                        "key" : key_,
                        "func_name" : func_name,
                        "func_info" : func_info,
                    })
                # return_data.append(func_info.get(embedding_type))
                count = count + 1
    print ("found centroids ", len(return_data), " functions with ", tool, embedding_type)
    json.dump(return_data, open(f"{tool}_fp_centroids.json", 'w'), indent=2)
    return return_data
extract_centroid_code('merged_fp_all_tools.json', 'slither', slither_centroid_indices, 'code_embedding')
extract_centroid_code('merged_fp_all_tools.json', 'mythril', mythril_centroid_indices, 'code_embedding')